# Libbitcoin BX: Payment Channel

In this BX tutorial, we will demonstrate a simple payment channel between two participants, Alice and Bob.

* 0) Wallet Setup for Alice & Bob
* 1) Opening a payment channel
* 2) Updating a payment channel
* 3) Broadcasting an old channel state & subsequent penalty transaction

In [281]:
bx seed | bx ec-new

7e52a59dc67f96a912001450b5f1b1309fdb95f9f4a58ee4fea43595fd6dbd83


In [282]:
bx ec-to-public dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb &&
bx ec-to-public 2d80969c3c10a85ce8a887686f9faad63d0b8bd05f6b5f4e22e425073c263129

02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5
027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a


In [283]:
bx sha256 02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5 | bx ripemd160 &&
bx sha256 027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a | bx ripemd160

eb8e87d55a0a313628d70603ab342dd7c380993e
8d950d086cf089d0bc29a0184faa7a448fd8c841


In [284]:
bx ec-to-address -v 111 02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5 &&
bx ec-to-address -v 111 027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a


n2zTqLKDrSF79wA7dni9mRbGQAP8ifMTbh
mtRa8Liyeygehg3b4JgEowjSCH9DqbR3pX


Spendable UXTOs:
```
TXID: e25dc4172478e5de9b90742f535a15c82ae086b957f47107ffc1ddd9f914a6e6
Index: 0
Amount: 130000000

TXID: fd5e45759b80ca83665149b90bd77e3dcb087601758dfd58f20532c80a93679c
Index: 0
Amount: 72267635

```

<hr style="border: 0.5px dashed #000;">

## 1) Opening the payment channel
<br>
<img src="payment_channel_funding.jpg" alt="drawing" style="" width="700px"/>

## 1.1) Alice creates the funding transaction

### 1.1.1) A + B multisig output script
* `2`
* `public key A`
* `public key B`
* `2`
* `checkmultisig`


In [293]:
bx script-encode \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 CHECKMULTISIG"


524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae


### 1.1.2) Minimum fee calculation
* `100 Satoshi/sigops + 1 Satoshi/Byte` 
* Multisig counts as `4 * 20 sigops`

In [295]:
expr 72267635 - 20 \* 4 \* 100 - 340

72259295


### 1.1.3) Encode funding transaction for signing.

In [23]:
bx tx-encode \
-i fd5e45759b80ca83665149b90bd77e3dcb087601758dfd58f20532c80a93679c:0 \
-o 524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae:72259295

01000000019c67930ac83205f258fd8d75017608cb3d7ed70bb949516683ca809b75455efd0000000000ffffffff01df964e040000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


### 1.1.4) Alice signs funding transaction.

In [24]:
bx fetch-tx -f json e25dc4172478e5de9b90742f535a15c82ae086b957f47107ffc1ddd9f914a6e6 \
| jq ".transaction.outputs[0].script"

"dup hash160 [eb8e87d55a0a313628d70603ab342dd7c380993e] equalverify checksig"


In [25]:
bx input-sign \
dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb \
"dup hash160 [eb8e87d55a0a313628d70603ab342dd7c380993e] equalverify checksig" \
01000000019c67930ac83205f258fd8d75017608cb3d7ed70bb949516683ca809b75455efd0000000000ffffffff01df964e040000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


3045022100d56c7408d1e89d5229fdc5fbe795d5b4c4f053f53a3f5c91b87814aed95b576d02207b1c9b16290cc6c7d3e68934908cef1e008ac8e2cfb5728c05273c38378f5d3201


In [26]:
bx input-set \
"[3045022100d56c7408d1e89d5229fdc5fbe795d5b4c4f053f53a3f5c91b87814aed95b576d02207b1c9b16290cc6c7d3e68934908cef1e008ac8e2cfb5728c05273c38378f5d3201] [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5]" \
01000000019c67930ac83205f258fd8d75017608cb3d7ed70bb949516683ca809b75455efd0000000000ffffffff01df964e040000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


01000000019c67930ac83205f258fd8d75017608cb3d7ed70bb949516683ca809b75455efd000000006b483045022100d56c7408d1e89d5229fdc5fbe795d5b4c4f053f53a3f5c91b87814aed95b576d02207b1c9b16290cc6c7d3e68934908cef1e008ac8e2cfb5728c05273c38378f5d32012102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5ffffffff01df964e040000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


In [27]:
bx validate-tx \
01000000019c67930ac83205f258fd8d75017608cb3d7ed70bb949516683ca809b75455efd000000006b483045022100d56c7408d1e89d5229fdc5fbe795d5b4c4f053f53a3f5c91b87814aed95b576d02207b1c9b16290cc6c7d3e68934908cef1e008ac8e2cfb5728c05273c38378f5d32012102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5ffffffff01df964e040000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


previous output not found


: 255

In [ ]:
bx tx-decode -f json \
01000000019c67930ac83205f258fd8d75017608cb3d7ed70bb949516683ca809b75455efd000000006b483045022100d56c7408d1e89d5229fdc5fbe795d5b4c4f053f53a3f5c91b87814aed95b576d02207b1c9b16290cc6c7d3e68934908cef1e008ac8e2cfb5728c05273c38378f5d32012102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5ffffffff01df964e040000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000 \
|jq ".transaction.hash"


**Important:** Alice can only broadcast the funding transaction once commitment 0 has been signed by both parties, as in the following section. Otherwise, her funds could become unspendable without Bob's cooperation. 

## 1.2) Commitment transactions `0a/0b`


### 1.2.1) Commitment transaction `0a`

**Output Scripts**
* RSMC A
* PKPKH B

**RSMC A**
* `IF`
    * `[public key R]`
* `ELSE`
    * `[3-byte delay]`
    * `checksequenceverify`
    * `drop`
    * `[public key A]`
* `ENDIF`
* `CHECKSIG`

**`Secret r = r_A + r_B`**
* Both Alice and Bob independently generate secrets `r_A` and `r_B`
* Alice and Bob only exchange public keys `R_A` AND `R_B`
* `public key R = R_A + R_B = r_A * G + r_B * G`

In [ ]:
bx ec-to-public fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5 &&
bx ec-to-public a7f98d6d45a96f94f314034535b2e5d86c659adde3e05d3b91b8c1a84b597ffd

In [ ]:
# Public key R
bx ec-add 031111a478bb24bd57e67ee3bacb920f2b38d6730ef9cd98f0386b4eea6eafee7a fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5

**Encoding Output Scripts for Commitment `0a`**

In [ ]:
#RSMC A
bx script-encode \
"IF [0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058] ELSE [050000] CHECKSEQUENCEVERIFY DROP [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5] ENDIF CHECKSIG"


In [ ]:
#P2PKH B
bx script-encode \
"DROP DUP HASH160 [8d950d086cf089d0bc29a0184faa7a448fd8c841] EQUALVERIFY CHECKSIG"


**Minimum Fee Calculation**

In [ ]:
expr 72267635 - 20 \* 4 \* 100 - 340

**Encode Commitment `0a` for signing**

In [ ]:
bx tx-encode \
-i b995a01a12d34e8bc4792946fb84c38ce5c656a6ea57c70747434281302dc5a1:0 \
-o 63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac:72258610 \
7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac: 200000


**Signing by Alice & Bob**

In [30]:
bx input-sign dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000 \
&&
bx input-sign 2d80969c3c10a85ce8a887686f9faad63d0b8bd05f6b5f4e22e425073c263129 \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000


304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01
3045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01


In [31]:
bx input-set \
"zero [304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01] [3045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01]" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000


0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b900000000920047304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01483045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000


**With commitment `0a` now complete, Alice is now ready to broadcast the funding transaction**

In [32]:
bx send-tx \
01000000019c67930ac83205f258fd8d75017608cb3d7ed70bb949516683ca809b75455efd000000006b483045022100d56c7408d1e89d5229fdc5fbe795d5b4c4f053f53a3f5c91b87814aed95b576d02207b1c9b16290cc6c7d3e68934908cef1e008ac8e2cfb5728c05273c38378f5d32012102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5ffffffff01df964e040000000087524104b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc41047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb252ae00000000


previous output not found


: 255

### 1.2.2) Commitment transaction `0b`
* Not shown
* Follows section 1.2.1

<hr style="border: 0.5px dashed #000;">


## 2) Updating the channel
<br>
<img src="payment_channel_update.jpg" alt="drawing" style="" width="700px"/>

### 2.1) New commitment transaction `1a`/`1b` are created
* Not shown 
* Follows section 1.2


### 2.2) Alice reveals `r_A` to Bob
* Since Alice is paying Bob, she must revoke her past commitment TX output. 
* Bob can now compute `r` = `r_A + r_B`

In [6]:
bx ec-add-secrets \
fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5 \
a7f98d6d45a96f94f314034535b2e5d86c659adde3e05d3b91b8c1a84b597ffd


a5dcdf16bc3afc2ff713cdc6ae3a78bdf36af636f1a084b065689612c9cefc71


<hr style="border: 0.5px dashed #000;">

## 3) Alice broadcasts old state. Bob sweeps the channel.
<br>
<img src="payment_channel_penalty.jpg" alt="drawing" style="" width="700px"/>

### 3.1) Alice broadcasts commitment tx `0a`

In [ ]:
bx send-tx 


### 3.2) Bob sweeps funds from commitment tx `0a`

#### 3.2.1) Encode penalty transaction for signing

In [ ]:
# Encode output spendable by Bob
bx script-encode \
"DROP DUP HASH160 [8d950d086cf089d0bc29a0184faa7a448fd8c841] EQUALVERIFY CHECKSIG"


In [ ]:
bx tx-encode \
-i b995a01a12d34e8bc4792946fb84c38ce5c656a6ea57c70747434281302dc5a1:0 \
b995a01a12d34e8bc4792946fb84c38ce5c656a6ea57c70747434281302dc5a1:1
-o 7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac:72258660


#### 3.2.2) Input script spending RSMC A

#### 3.2.3) Input script spending P2PKH B

In [ ]:
bx script-encode \
"DROP DUP HASH160 [eb8e87d55a0a313628d70603ab342dd7c380993e] EQUALVERIFY CHECKSIG"

In [ ]:
expr 72259295 - 1 \* 4 \* 100 - 235

In [207]:
bx tx-encode \
-i b995a01a12d34e8bc4792946fb84c38ce5c656a6ea57c70747434281302dc5a1:0 \
-o 7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac:72258660

0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000


In [208]:
bx fetch-tx -f json b995a01a12d34e8bc4792946fb84c38ce5c656a6ea57c70747434281302dc5a1 \
| jq ".transaction.outputs[0].script"

"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig"


TODO: Sighash none for B signature.

In [296]:
bx input-sign dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000 \
&&
bx input-sign 2d80969c3c10a85ce8a887686f9faad63d0b8bd05f6b5f4e22e425073c263129 \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000


304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01
3045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01


In [297]:
bx input-set \
"zero [304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01] [3045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01]" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000


0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b900000000920047304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01483045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000


In [298]:
bx validate-tx 0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b900000000920047304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01483045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000

previous output not found


: 255

In [299]:
bx tx-decode -f json \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b900000000920047304402206d6c21404b467246d2ce6177140fa6ce530c8e18db93acece9577e6bc2b6cfc1022016b610ced9bfab67709bfab3ac5ea24034dbd4a842c76e446d6bf5de77b3876e01483045022100fae26bce880ee44ebfbed8dc9c6455a01bacc025cbef3787b9a4d438a10d74cf022045d12fd6cf15601dcc246da119a9f1991d2a853306c25ec72168f90659c9324a01ffffffff0164944e04000000001a7576a914eb8e87d55a0a313628d70603ab342dd7c380993e88ac00000000 \
|jq ".transaction.hash"

"605083d0800f16112f1627f96af928cbff373fc108a37801fea9e913f738d39d"


<hr style="border: 0.5px dashed #000;">

Generate new R Keys:

In [300]:
bx ec-to-public fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5 &&
bx ec-to-public a7f98d6d45a96f94f314034535b2e5d86c659adde3e05d3b91b8c1a84b597ffd

032377c6bb0ab686e3f01aa30c8dc6e5711814cde5bd2d3b9a609f39a4528da783
031111a478bb24bd57e67ee3bacb920f2b38d6730ef9cd98f0386b4eea6eafee7a


In [301]:
bx ec-add 031111a478bb24bd57e67ee3bacb920f2b38d6730ef9cd98f0386b4eea6eafee7a fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5

0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058


In [302]:
bx ec-add-secrets fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5 a7f98d6d45a96f94f314034535b2e5d86c659adde3e05d3b91b8c1a84b597ffd \
| bx ec-to-public

0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058


Create RSMC A or B:
* IF
    * [public key R]
* ELSE
    * [3-byte delay]
    * checksequenceverify
    * drop
    * [public key A or B]
* ENDIF
* CHECKSIG
    

In [303]:
bx script-encode \
"IF [0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058] ELSE [050000] CHECKSEQUENCEVERIFY DROP [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5] ENDIF CHECKSIG"


63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac


In [304]:
bx script-encode \
"IF [0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058] ELSE [050000] CHECKSEQUENCEVERIFY DROP [027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a] ENDIF CHECKSIG"


63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b27521027546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a68ac


In [221]:
bx fetch-tx -f json b995a01a12d34e8bc4792946fb84c38ce5c656a6ea57c70747434281302dc5a1 \
| jq ".transaction.outputs[0].script"

"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig"


In [239]:
expr 72259295 - 1 \* 4 \* 100 - 285

72258610


In [240]:
bx tx-encode \
-i b995a01a12d34e8bc4792946fb84c38ce5c656a6ea57c70747434281302dc5a1:0 \
-o 63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac:72258610


0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000


In [241]:
bx input-sign dbafbb58c0e58a0609898e831d387e7ed63275e21c5755f1b4600b36cb314cbb \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000 \
&&
bx input-sign 2d80969c3c10a85ce8a887686f9faad63d0b8bd05f6b5f4e22e425073c263129 \
"2 [04b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc564bec969580738901a21e5b5dbd687ed9897a5a01a9e99a8dda825e14beee4bc] [047546622c89d92aa58ad5cc0127dd67bac986c2fb52abfa4ed493ca9fa73e2f2a071bfd964f6ea4d9a0514db400a5df56a6fd5a058b4ea7444b7df8d834637fb2] 2 checkmultisig" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000


3045022100f466fa55a0bf09a76a9cd5dfcecd70eb60da5f1171a05e0e877ca04c474a644c02200c83421d4b0827da1c5e4cfd7d2857ce84c1616d9abca39f5d68a0a117d55a9d01
3045022100c9526668d57cfe41fba009e70da756c5152b4a1b6ab7307ff7b68cea19a6082f0220354be51a646bd95b529b497d48188ba5cdf68439a10ac11072fff192a7921c0a01


In [242]:
bx input-set \
"zero [3045022100f466fa55a0bf09a76a9cd5dfcecd70eb60da5f1171a05e0e877ca04c474a644c02200c83421d4b0827da1c5e4cfd7d2857ce84c1616d9abca39f5d68a0a117d55a9d01] [3045022100c9526668d57cfe41fba009e70da756c5152b4a1b6ab7307ff7b68cea19a6082f0220354be51a646bd95b529b497d48188ba5cdf68439a10ac11072fff192a7921c0a01]" \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b90000000000ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000


0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b9000000009300483045022100f466fa55a0bf09a76a9cd5dfcecd70eb60da5f1171a05e0e877ca04c474a644c02200c83421d4b0827da1c5e4cfd7d2857ce84c1616d9abca39f5d68a0a117d55a9d01483045022100c9526668d57cfe41fba009e70da756c5152b4a1b6ab7307ff7b68cea19a6082f0220354be51a646bd95b529b497d48188ba5cdf68439a10ac11072fff192a7921c0a01ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000


In [305]:
bx validate-tx \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b9000000009300483045022100f466fa55a0bf09a76a9cd5dfcecd70eb60da5f1171a05e0e877ca04c474a644c02200c83421d4b0827da1c5e4cfd7d2857ce84c1616d9abca39f5d68a0a117d55a9d01483045022100c9526668d57cfe41fba009e70da756c5152b4a1b6ab7307ff7b68cea19a6082f0220354be51a646bd95b529b497d48188ba5cdf68439a10ac11072fff192a7921c0a01ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000


matching transaction with unspent outputs


: 255

In [ ]:
bx send-tx \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b9000000009300483045022100f466fa55a0bf09a76a9cd5dfcecd70eb60da5f1171a05e0e877ca04c474a644c02200c83421d4b0827da1c5e4cfd7d2857ce84c1616d9abca39f5d68a0a117d55a9d01483045022100c9526668d57cfe41fba009e70da756c5152b4a1b6ab7307ff7b68cea19a6082f0220354be51a646bd95b529b497d48188ba5cdf68439a10ac11072fff192a7921c0a01ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000

In [306]:
bx tx-decode -f json \
0100000001a1c52d308142434707c757eaa656c6e58cc384fb462979c48b4ed3121aa095b9000000009300483045022100f466fa55a0bf09a76a9cd5dfcecd70eb60da5f1171a05e0e877ca04c474a644c02200c83421d4b0827da1c5e4cfd7d2857ce84c1616d9abca39f5d68a0a117d55a9d01483045022100c9526668d57cfe41fba009e70da756c5152b4a1b6ab7307ff7b68cea19a6082f0220354be51a646bd95b529b497d48188ba5cdf68439a10ac11072fff192a7921c0a01ffffffff0132944e04000000004e63210358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed0440586703050000b2752102b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc568ac00000000 \
|jq ".transaction.hash"

"6c3f13be372fe3337e2ca2ae1bf19f14a8b7e214e085725553bdc6d8cbe922cb"


<hr style="border: 0.5px dashed #000;">

Spend RSMC A:
* After timeout:
    * `[Sig A]`
    * `0`
* Penalty spend if revoked:
    * `[Sig R]`
    * `1`

In [307]:
bx ec-add-secrets fde351a976918c9b03ffca81788792e441b4383fbd08c7b0938232f74eabbdb5 a7f98d6d45a96f94f314034535b2e5d86c659adde3e05d3b91b8c1a84b597ffd

a5dcdf16bc3afc2ff713cdc6ae3a78bdf36af636f1a084b065689612c9cefc71


In [4]:
bx ec-to-public a5dcdf16bc3afc2ff713cdc6ae3a78bdf36af636f1a084b065689612c9cefc71

0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058


In [308]:
# Sends funds to B public key hash.
bx script-encode \
"DROP DUP HASH160 [8d950d086cf089d0bc29a0184faa7a448fd8c841] EQUALVERIFY CHECKSIG"

7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac


In [309]:
expr 72258610 - 1 \* 4 \* 100 - 285

72257925


In [310]:
bx tx-encode \
-i 6c3f13be372fe3337e2ca2ae1bf19f14a8b7e214e085725553bdc6d8cbe922cb:0 \
-o 7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac:72257925


0100000001cb22e9cbd8c6bd53557285e014e2b7a8149ff11baea22c7e33e32f37be133f6c0000000000ffffffff0185914e04000000001a7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


In [274]:
bx fetch-tx -f json 6c3f13be372fe3337e2ca2ae1bf19f14a8b7e214e085725553bdc6d8cbe922cb \
| jq ".transaction.outputs[0].script"

"if [0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058] else [050000] checksequenceverify drop [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5] endif checksig"


In [275]:
bx input-sign a5dcdf16bc3afc2ff713cdc6ae3a78bdf36af636f1a084b065689612c9cefc71 \
"if [0358efa6bb04acb2c86342fa3dd84c2b360b1e8597b908a142ecca7517ed044058] else [050000] checksequenceverify drop [02b3b4a1b2bd5555cbe37cc8cd58d143ee93cdde65ac987c5d2fbcdfd6fe79ccc5] endif checksig" \
0100000001cb22e9cbd8c6bd53557285e014e2b7a8149ff11baea22c7e33e32f37be133f6c0000000000ffffffff0185914e04000000001a7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


3045022100aef0f728c921e318424e2b135b245a4a1e273abf6303a50f160305050b3ed6e0022022908996fe0e1ecfb324a9de52da1f94baba551ccfc372d431a416fa82583db601


In [2]:
bx input-set \
"[3045022100aef0f728c921e318424e2b135b245a4a1e273abf6303a50f160305050b3ed6e0022022908996fe0e1ecfb324a9de52da1f94baba551ccfc372d431a416fa82583db601] 1" \
0100000001cb22e9cbd8c6bd53557285e014e2b7a8149ff11baea22c7e33e32f37be133f6c0000000000ffffffff0185914e04000000001a7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


0100000001cb22e9cbd8c6bd53557285e014e2b7a8149ff11baea22c7e33e32f37be133f6c000000004a483045022100aef0f728c921e318424e2b135b245a4a1e273abf6303a50f160305050b3ed6e0022022908996fe0e1ecfb324a9de52da1f94baba551ccfc372d431a416fa82583db60151ffffffff0185914e04000000001a7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


In [5]:
bx validate-tx \
0100000001cb22e9cbd8c6bd53557285e014e2b7a8149ff11baea22c7e33e32f37be133f6c000000004a483045022100aef0f728c921e318424e2b135b245a4a1e273abf6303a50f160305050b3ed6e0022022908996fe0e1ecfb324a9de52da1f94baba551ccfc372d431a416fa82583db60151ffffffff0185914e04000000001a7576a9148d950d086cf089d0bc29a0184faa7a448fd8c84188ac00000000


The transaction is valid.
